In [2]:
print("Um cuspidor esta nascendo!")

Um cuspidor esta nascendo!


In [48]:
!pip install gradio duckdb pandas unidecode --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 5.7 MB/s eta 0:00:00


In [90]:
import traceback

class SpitterDucklakeOperator():
    def __init__(self,con,  path=None):

      self.ducklake_db = "spitter_ducklake"
      self.path = path if path else f"/tmp/dklq_{self.ducklake_db}"
      self.con = con
      self.init_ducklake()


    def init_ducklake(self):
      try:
        #con = duckdb.connect(database=':memory:')
        self.con.execute("INSTALL ducklake;")
        self.con.execute(f"DETACH DATABASE IF EXISTS {self.ducklake_db};")
        self.con.execute(f"ATTACH 'ducklake:metadata.ducklake' AS {self.ducklake_db} (DATA_PATH '{self.path}');")
        #return con
      except Exception as e:
        raise e

    def table_exist(self, table):
      try:
        con.execute(f"SELECT * FROM {table} LIMIT 0")
        return True
      except Exception as e:
        return False

    def normalize_column_name(self, col):
      col = unidecode(col).lower().strip()
      col = re.sub(r'[^a-z0-9_]', '_', col)
      col = re.sub(r'_+', '_', col)
      return col.strip('_')


    def create_or_replace_table(self, source_table_name, output_table_name, layer, primary_key=False):
      try:
        self.con.execute(f"CREATE SCHEMA IF NOT EXISTS {self.ducklake_db}.{layer}")
        if primary_key:
          instruction_sql = ','.join(["{0} {1} {2}".format(
                                                            self.normalize_column_name(row[0]),
                                                            row[1],
                                                            "" if row[0] != primary_key else 'PRIMARY KEY'
                                                          ) for row in con.execute(f"DESCRIBE FROM {source_table_name}").fetchall()
                                    ]
                                     )

          con.execute(f"""CREATE OR REPLACE TABLE {self.ducklake_db}.{layer}.{output_table_name} ({instruction_sql})""")
        else:
          self.con.execute(f"CREATE OR REPLACE TABLE {self.ducklake_db}.{layer}.{output_table_name} SELECT * FROM {source_table_name}")
        return True, "success"
      except Exception as e:
        return False, f"{e} _> {traceback.format_exc()}"

    def insert_table(self, source_table_name, output_table_name, layer,op_mode, primary_key=False):
      try:
        if not self.table_exist(f"{self.ducklake_db}.{layer}.{output_table_name}") or op_mode == "overwrite":
            self.create_or_replace_table(source_table_name, output_table_name, layer, primary_key)

        if primary_key:
          con.execute(f"INSERT OR REPLACE INTO {self.ducklake_db}.{layer}.{output_table_name} SELECT * FROM {source_table_name}")
        else:
          self.con.execute(f"INSERT INTO {self.ducklake_db}.{layer}.{output_table_name} SELECT * FROM {source_table_name}")

        return True, "success"
      except Exception as e:
        return False, f"{e} _> {traceback.format_exc()}"



In [100]:
#csv operations
import chardet
import duckdb
import pandas as pd
import traceback

class SpitterOperator():
    def __init__(self, file):
        self.df = pd.DataFrame()
        self.primary_key = ""
        self.cols = []
        self.con = duckdb.connect(database=':memory:')
        self.dklq = SpitterDucklakeOperator(self.con)
        self.table_name = ""
        self.op_mode = ""
        if file is not None:
            self.load_bronze(file)
        else:
            return "Envie um file CSV."

    def preview_bronze(self):
        norm_cols = ",".join([f'"{col}" AS {self.dklq.normalize_column_name(col)}' for col in self.cols])
        print("#######################",f"SELECT {norm_cols} FROM temp.tmp_{self.table_name} LIMIT 10")
        preview_df = self.con.execute(f"""
          SELECT {norm_cols} FROM tmp_{self.table_name} LIMIT 10
        """).fetchdf()
        return preview_df

    def set_primary_key(self, column_name):
        if column_name in self.df.columns:
            self.primary_key = column_name

    def set_table_op_mode(self, op_mode):
        self.op_mode = op_mode

    def persist_table_on_dklq(self, layer):
        result = self.dklq.insert_table(f"tmp_{self.table_name}", f"tb_{layer}_{self.table_name}", layer, self.op_mode, self.primary_key)
        if not result[0]:
          raise Exception(result[1])


    def load_bronze(self,file):

        try:
          self.table_name = file.name.split("/")[-1].split(".")[0]
          self.con.execute(f"""
              CREATE TEMPORARY TABLE tmp_{self.table_name} AS
              SELECT * FROM read_csv_auto('{file.name}', header=True)
          """)
          self.cols = [row[0] for row in self.con.execute(f"DESCRIBE tmp_{self.table_name}").fetchall()]
        except Exception as e:
          raise (e, traceback.format_exc())



In [99]:
con.execute("""
SELECT ID_compra AS id_compra,
filial AS filial,
cidade AS cidade,
tipo_cliente AS tipo_cliente,
genero AS genero,
tipo_produto AS tipo_produto,
preco_unitario AS preco_unitario,
quantidade AS quantidade,
'imposto_5%' AS imposto_5,
total AS total,
data AS data,
hora AS hora,
forma_pagamento AS forma_pagamento,
avaliacao AS avaliacao FROM temp.tmp_vendas LIMIT 10""")

CatalogException: Catalog Error: Table with name tmp_vendas does not exist!
Did you mean "my_ducklake.bronze.tb_bronze_vendas"?

LINE 15: avaliacao AS avaliacao FROM temp.tmp_vendas LIMIT 10
                                     ^

In [102]:
import gradio as gr

def init_file_operator(file):
    spitter_operator = SpitterOperator(file)
    return spitter_operator, "bronze"

def preview_bronze(spitter_operator):
    return spitter_operator.preview_bronze()

def set_pk_column(spitter_operator, pk_column):
    if spitter_operator is not None:
        spitter_operator.set_primary_key(pk_column)
    return spitter_operator

#button options
def set_pk_chk_visibility(spitter_operator, checkbox_value):
    if checkbox_value:
        return gr.update(interactive=True, visible=True,choices=spitter_operator.cols)
    else:
        return gr.update(interactive=False, visible=False)

def set_op_chk_visibility(spitter_operator, layer):
    choices = ["append", "overwrite"] if layer == "bronze" else ["overwrite","upsert"]
    return gr.update(interactive=True, visible=True, choices= choices)


def set_table_op_mode_btn(spitter_operator, op_mode):
  spitter_operator.set_table_op_mode(op_mode)
  return f"Commit - table -> {spitter_operator.table_name} | op_mode -> {op_mode}", True

def publish_bronze_table(spitter_operator, commit_state, layer):
  if commit_state:
    spitter_operator.persist_table_on_dklq(layer)
    return "sucess"
  else:
    return "Error. Please check your settings!"




with gr.Blocks(title="# Arquitetura Medalhão - Data Spitter - CSV - multi Reads") as demo:
    gr.Markdown("# Arquitetura Medalhão (Bronze / Silver / Gold)")
    gr.Markdown("## Spitter Data Lakehouse Explorer")
    gr.Markdown("## Leia seus dados, configure as operacoes, e maos a obra. Apresentamos suporte a cargas incrementais e de sobrescrita")
    gr.Markdown("As tabelas criadas utilizarao o nome do arquivo como referencia")
    st_spitter_operator = gr.State(None)
    st_layer = gr.State("")

    #BronzeLayer
    with gr.Tab("Bronze Layer"):
        gr.Markdown("### Upload dos dados brutos")
        st_choices = gr.State([])
        st_commit_bronze = gr.State(False)

        csv_bronze = gr.File(label="Selecione um CSV",file_count="single", type="filepath")
        csv_bronze.upload(init_file_operator, inputs=csv_bronze, outputs=[st_spitter_operator,st_layer])

        gr.Markdown("### Preview dos dados brutos")
        bronze_btn = gr.Button("Preview Bronze")
        bronze_table = gr.DataFrame(label="Bronze Data")
        bronze_btn.click(preview_bronze, inputs=st_spitter_operator, outputs=bronze_table)

        gr.Markdown("### Limpeza dos dados brutos")


        op_dropdown = gr.Dropdown(label="Select the table operaration mode (required)", choices=[], interactive=False, visible=False)
        st_spitter_operator.change(set_op_chk_visibility,inputs=[st_spitter_operator, st_layer],outputs=op_dropdown)

        output_text = gr.Markdown("")
        op_dropdown.change(set_table_op_mode_btn, inputs=[st_spitter_operator, op_dropdown], outputs=[output_text, st_commit_bronze])

        alerta_md_component = gr.Markdown("")
        publish_bronze_btn = gr.Button(value=f"Commit", visible=True)
        publish_bronze_btn.click(publish_bronze_table, inputs=[st_spitter_operator, st_commit_bronze, st_layer], outputs=alerta_md_component)
    #SilverLayer
    with gr.Tab("Camada Silver"):
        gr.Markdown("### Limpeza dos dados brutos")
        pk_checkbox = gr.Checkbox(label="Does the table have a primary key?", value=False)
        pk_dropdown = gr.Dropdown(label="Select the primary key", choices=[], interactive=False, visible=False)
        pk_checkbox.change(set_pk_chk_visibility, inputs=[st_spitter_operator, pk_checkbox], outputs=pk_dropdown)
        pk_dropdown.change(set_pk_column, inputs=[st_spitter_operator, pk_dropdown], outputs=st_spitter_operator)



demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2cb0f36e1702f7ff01.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2cb0f36e1702f7ff01.gradio.live
